# Import the Required Packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

# Data pre-processing

## Read the data

In [ ]:
# Define the file path for the CSV file
file_path = '/path/to/your/df_final.csv'  # Adjust the path accordingly

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame to get an overview of the data
display(df.head())

# Print out detailed information about the DataFrame, such as column names, data types, and non-null counts
print(df.info())

## Handle missing values, duplicates, and reason extraction failures

In [ ]:
# Check and extract rows with missing values
df_missing = df[df.isnull().any(axis=1)]

# Check and extract rows with duplicate "decision_id"
df_duplicates = df[df.duplicated(subset=['decision_id'])]

# Identify rows where all reason columns (reason_1 to reason_7) are zero
reasons_columns = [
    'reason_1', 'reason_2', 'reason_3', 'reason_4', 'reason_5', 'reason_6',
    'reason_7'
]
zero_reasons_df = df[(df[reasons_columns] == 0).all(axis=1)]

# Print the count of rows with missing values, duplicate rows, and rows with all zero reasons
missing_values_count = df.isnull().any(axis=1).sum()
duplicate_rows_count = df.duplicated().sum()
zero_reasons_count = zero_reasons_df.shape[0]

print(
    f"Number of missing values: {missing_values_count}, accounting for {(missing_values_count/len(df)) * 100:.2f}% of the total data."
)
print(
    f"Number of duplicate rows: {duplicate_rows_count}, accounting for {(duplicate_rows_count/len(df)) * 100:.2f}% of the total data."
)
print(
    f"Number of rows where all reason columns are zero: {zero_reasons_count}, accounting for {(zero_reasons_count/len(df)) * 100:.2f}% of the total data."
)

# Remove rows with missing values
df_cleaned = df.dropna()

# Remove duplicate rows, keeping only one instance of each "decision_id"
df_cleaned = df_cleaned.drop_duplicates()

# Add a new column 'reason_non' to indicate rows where all reason columns are zero
df_cleaned['reason_non'] = ((df_cleaned[reasons_columns] == 0).all(axis=1)).astype(int)

# Insert 'reason_non' column between 'reason_7' and 'background'
df_cleaned.insert(df_cleaned.columns.get_loc('background'), 'reason_non', df_cleaned.pop('reason_non'))

# Define a function to count the number of words in a string
def word_count(text):
    return len(text.split())

# Apply the function and filter rows with less than 50 words
df_cleaned = df_cleaned[df_cleaned['background'].apply(word_count) >= 50]

# Optional: Visualize the missing values
# plt.figure(figsize=(12, 6))
# sns.heatmap(df.isnull(), cbar=False, cmap='viridis')
# plt.title('Heatmap of Missing Values')
# plt.show()

# Optional: Visualize the duplicate rows
# duplicate_counts = df.duplicated().value_counts()
# plt.figure(figsize=(8, 6))
# sns.barplot(x=duplicate_counts.index, y=duplicate_counts.values, palette='viridis')
# plt.title('Count of Duplicate Rows')
# plt.xlabel('Is Duplicate')
# plt.ylabel('Count')
# plt.show()

# Optional: Display DataFrames with missing values and duplicates
# display(df_missing)
# display(df_duplicates)
# print("DataFrame with Missing Values:")
# print(df_missing)
# print("DataFrame with Duplicates:")
# print(df_duplicates)

## Processing "date" data

In [ ]:
# Convert the 'date' column to datetime format
df_cleaned['date'] = pd.to_datetime(df_cleaned['date'])

## Processing "decision", "gender" and "reason" data

In [ ]:
# Convert 'decision', 'gender', and 'reason' columns to integer type
cols_to_convert = ['decision', 'gender', 'reason_1', 'reason_2', 'reason_3', 'reason_4', 'reason_5', 'reason_6', 'reason_7']
df_cleaned[cols_to_convert] = df_cleaned[cols_to_convert].astype(int)

## Processing "category" data

In [ ]:
# Ensure the 'category' column is treated as a categorical variable
df_cleaned['category'] = df_cleaned['category'].astype('category')

## Processing "compensation" data

In [ ]:
# Define a function to process the 'compensation' column
def process_compensation(compensation):
    """
    Process the compensation value:
    - If it's 'Non-pecuniary compensation', return it as is.
    - If it contains a currency symbol (£), extract and clean the numeric part.
    - For any other text, remove non-alphanumeric characters.
    
    Parameters:
    compensation (str): The original compensation text.
    
    Returns:
    str or float: Cleaned compensation value as float if numeric, otherwise the cleaned text.
    """
    # Return immediately if it's a non-pecuniary compensation
    if compensation == "Non-pecuniary compensation":
        return compensation
    
    # Extract and clean numeric compensation if it contains a currency symbol
    if "£" in compensation:
        # Remove all characters that are not digits or decimal points
        compensation = re.sub(r'[^\d.]', '', compensation)
        # Handle cases with multiple decimal points, keep only the first valid number
        if compensation.count('.') > 1:
            compensation = compensation.split('.')[0] + '.' + compensation.split('.')[1]
        return float(compensation)
    
    # Remove all other symbols and text for non-numeric compensations
    return re.sub(r'[^\w\s]', '', compensation).strip()

# Apply the compensation processing function to the 'compensation' column
df_cleaned['compensation'] = df_cleaned['compensation'].apply(process_compensation)

## Add "include": 0 means SB doesn't own this category; 1 means SB owns this category

In [ ]:
# Create a mapping dictionary for categories and inclusion flags
mapping = {
    "category": ["c1", "c2", "c3", "c4", "c5", "c6", "c7", "c8", "c9", "c10", "c11"],
    "include": [0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1]
}

# Convert the mapping dictionary to a DataFrame
df_mapping = pd.DataFrame(mapping)

# Merge the cleaned DataFrame with the mapping DataFrame on the 'category' column
df_processed = pd.merge(df_cleaned, df_mapping, on="category", how="left")

## Output the processed dataset

In [ ]:
df_processed.to_csv('path/to/save/data_extracted.csv', index=False)

# Code for Insurance Product Complaint Case Analysis Section

In [ ]:
# Load the CSV file into a DataFrame
file_path = 'path/to/save/data_extracted.csv'
df_eda = pd.read_csv(file_path)

## Overall Situation

In [ ]:
# Convert the 'date' column to datetime format
df_eda['date'] = pd.to_datetime(df_eda['date'])

# Ensure that the 'category' column is treated as a categorical variable
df_eda['category'] = df_eda['category'].astype('category')

# Define the columns that record reasons for complaints
reasons = ['reason_1', 'reason_2', 'reason_3', 'reason_4', 'reason_5', 'reason_6', 'reason_7', 'reason_non']

# Set the color palette for visualizations using a HSV (Hue, Saturation, Value) color scheme
colors = sns.color_palette("hsv", len(reasons))

# Define the order for sorting insurance product categories
categories = ['c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10', 'c11']

### 1. Distribution of uphold rate (decision variable being 1) over time

In [ ]:
# Set font sizes for titles and axis labels
plt.rcParams.update({'axes.titlesize': 55, 'axes.labelsize': 45, 'xtick.labelsize': 40, 'ytick.labelsize': 40})

# Set the background color of the plots to a light gray
sns.set_style("whitegrid", {"axes.facecolor": ".3"})

# Filter the DataFrame to include only numeric and datetime columns
df_eda_numeric = df_eda.select_dtypes(include=['number', 'datetime64'])

# 1. Distribution of uphold rate (where the 'decision' variable is 1) over time, with daily precision
df_eda_daily = df_eda_numeric.set_index('date').resample('D').mean().reset_index()
df_eda_daily['decision_rate'] = df_eda_daily['decision'].rolling(window=30, min_periods=1).mean()

# Create a figure for the plot
plt.figure(figsize=(30, 16))
sns.lineplot(data=df_eda_daily, x='date', y='decision_rate', legend=False, linewidth=7)  # Set line width

# Add title and axis labels
plt.title('Uphold Rate Over Time', pad=30)  # The pad parameter adds space between the title and the plot
plt.xlabel('Date')
plt.ylabel('Uphold Rate')

# Set y-axis limits and add a grid to the plot
plt.ylim(0, 1)
plt.grid(True)
plt.xticks(rotation=45)

# Set a general save path for the figure (user should adjust the path as needed)
save_path = 'path/to/save/uphold_rate_over_time.png'

# Save the plot as an image file
plt.savefig(save_path)

# Display the plot
plt.show()

### Distribution of total number of complaints over time

In [ ]:
# Set font sizes for titles and axis labels
plt.rcParams.update({'axes.titlesize': 55, 'axes.labelsize': 45, 'xtick.labelsize': 40, 'ytick.labelsize': 40})

# Set the background color of the plots to a light gray
sns.set_style("whitegrid", {"axes.facecolor": ".3"})

# Calculate the total number of complaints per month
df_eda_daily_count = df_eda.set_index('date').resample('M').size().reset_index(name='complaint_count')

# Create a figure for the plot
plt.figure(figsize=(30, 16))
sns.lineplot(data=df_eda_daily_count, x='date', y='complaint_count', legend=False, linewidth=7)  # Set line width

# Add title and axis labels
plt.title('Total Complaints Over Time', pad=30)  # The pad parameter adds space between the title and the plot
plt.xlabel('Date')
plt.ylabel('Total Complaints')

# Add a grid to the plot and rotate x-axis labels
plt.grid(True)
plt.xticks(rotation=45)

# Set a general save path for the figure (user should adjust the path as needed)
save_path = 'path/to/save/total_complaints_over_time.png'

# Save the plot as an image file
plt.savefig(save_path)

# Display the plot
plt.show()

### 2. Distribution of different complained product categories over time

In [ ]:
# Distribution of Product Categories Over Time (Monthly)
# Set background color to light gray
sns.set_style("whitegrid", {"axes.facecolor": ".3"})

# Set title and axis label font sizes
plt.rcParams.update({
    'axes.titlesize': 35, 
    'axes.labelsize': 30, 
    'xtick.labelsize': 25, 
    'ytick.labelsize': 20
})

# Set up subplots
fig, axs = plt.subplots(4, 3, figsize=(30, 16), sharex=True, sharey=True)
colors = sns.color_palette("hsv", len(categories))

# Group data by month and category, then calculate counts
df_category_counts_monthly = df_eda.groupby([pd.Grouper(key='date', freq='M'), 'category'], observed=True).size().reset_index(name='count')

# Calculate total number of complaints per month
df_category_counts_monthly['total_count'] = df_category_counts_monthly.groupby('date')['count'].transform('sum')

# Calculate the proportion of each product category per month
df_category_counts_monthly['proportion'] = df_category_counts_monthly['count'] / df_category_counts_monthly['total_count']

# Plot each category's proportion over time
for ax, category, color in zip(axs.flat, categories, colors):
    category_data = df_category_counts_monthly[df_category_counts_monthly['category'] == category]
    sns.lineplot(data=category_data, x='date', y='proportion', ax=ax, color=color, linewidth=6)
    ax.set_title(f'Category {category}')
    ax.set_xlabel('Date')
    ax.set_ylabel('Proportion')
    ax.grid(True)  # Add grid to each subplot
    for label in ax.get_xticklabels():
        label.set_rotation(45)

# Hide any unused subplots
for i in range(len(categories), len(axs.flat)):
    fig.delaxes(axs.flat[i])

# Set main title and adjust subplot layout
plt.tight_layout(rect=[0, 0, 1, 0.93])  # Adjust subplot layout to avoid overlapping the title
fig.suptitle('Proportion of Complained Product Categories Over Time (Monthly)', fontsize=40)

# Set a general save path for the figure (user should adjust the path as needed)
save_path = 'path/to/save/proportion_of_product_categories_over_time_monthly_subplots.png'

# Save the plot as an image file
plt.savefig(save_path)

# Display the plot
plt.show()

### 3. Distribution of different complaint reasons over time

In [ ]:
# Set the background color to light gray
sns.set_style("whitegrid", {"axes.facecolor": ".3"})

# Set font sizes for titles and axis labels
plt.rcParams.update({
    'axes.titlesize': 35, 
    'axes.labelsize': 30, 
    'xtick.labelsize': 25, 
    'ytick.labelsize': 20
})

# Distribution of Complaint Reasons as a Percentage Over Time (Monthly)
fig, axs = plt.subplots(4, 2, figsize=(30, 16), sharex=True, sharey=True)

# Convert 'count' to proportion of total complaints per month
df_reason_counts = df_eda.melt(id_vars=['date'], value_vars=reasons, var_name='reason', value_name='count')
df_reason_counts['count'] = df_reason_counts['count'].astype(int)
df_reason_counts_monthly = df_reason_counts.groupby([pd.Grouper(key='date', freq='M'), 'reason'], observed=True).sum().reset_index()

# Calculate total complaints per month
df_total_monthly = df_reason_counts_monthly.groupby('date')['count'].sum().reset_index()
df_reason_counts_monthly = pd.merge(df_reason_counts_monthly, df_total_monthly, on='date', suffixes=('', '_total'))

# Calculate proportion of each reason
df_reason_counts_monthly['proportion'] = df_reason_counts_monthly['count'] / df_reason_counts_monthly['count_total']

# Plot each reason's proportion over time
for ax, reason, color in zip(axs.flat, reasons, colors):
    reason_data = df_reason_counts_monthly[df_reason_counts_monthly['reason'] == reason]
    sns.lineplot(data=reason_data, x='date', y='proportion', ax=ax, color=color, linewidth=6)
    ax.set_title(f'{reason}')
    ax.set_xlabel('Date')
    ax.set_ylabel('Proportion')
    ax.grid(True)  # Add grid to each subplot
    for label in ax.get_xticklabels():
        label.set_rotation(45)

# Hide any unused subplots
for i in range(len(reasons), len(axs.flat)):
    fig.delaxes(axs.flat[i])

# Set the main title and adjust layout to avoid overlapping
plt.tight_layout(rect=[0, 0, 1, 0.93])
fig.suptitle('Proportion of Complaint Reasons Over Time (Monthly)', fontsize=40)

# Specify the save path (user should adjust the path as needed)
save_path = 'path/to/save/proportion_of_complaint_reasons_over_time_monthly_subplots.png'

# Save the plot as an image file
plt.savefig(save_path)

# Display the plot
plt.show()

### 4. Correlation analysis and visualization

In [ ]:
# # Handling 'category' as categorical and encoding it for correlation
# df_eda_encoded = df_eda.copy()
# df_eda_encoded['category'] = df_eda_encoded['category'].astype('category').cat.codes

# # Recompute correlation matrix with encoded categories
# corr_matrix_encoded = df_eda_encoded[['decision', 'gender', 'category', 'reason_1', 'reason_2', 'reason_3', 'reason_4', 'reason_5', 'reason_6', 'reason_7']].corr()

# plt.figure(figsize=(10, 8))
# sns.heatmap(corr_matrix_encoded, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
# plt.title('Correlation Matrix with Encoded Categories')
# plt.tight_layout()
# plt.show()

### 5. Uphold rate for each product category over time

In [ ]:
# Set background color to light gray
sns.set_style("whitegrid", {"axes.facecolor": ".3"})

# Set font sizes for titles and axis labels
plt.rcParams.update({
    'axes.titlesize': 35, 
    'axes.labelsize': 30, 
    'xtick.labelsize': 25, 
    'ytick.labelsize': 20,
    'legend.facecolor': 'white'  # Set legend background color to white
})

# Calculate the overall uphold rate baseline (Monthly)
df_eda_numeric = df_eda.select_dtypes(include=['number', 'datetime64'])
df_eda_monthly = df_eda_numeric.set_index('date').resample('M').mean().reset_index()
df_eda_monthly['decision_rate'] = df_eda_monthly['decision'].rolling(window=6, min_periods=1).mean()

# Set up subplots for each product category (Monthly)
fig, axs = plt.subplots(4, 3, figsize=(30, 16), sharex=True, sharey=True)

# Resample decision rate by category and date (Monthly)
df_eda_numeric_category = df_eda[['date', 'category', 'decision']].copy()
df_eda_monthly_category = df_eda_numeric_category.set_index('date').groupby('category', observed=True).resample('M').mean(numeric_only=True).reset_index()
df_eda_monthly_category['decision_rate'] = df_eda_monthly_category.groupby('category', observed=True)['decision'].transform(lambda x: x.rolling(window=6, min_periods=1).mean())

# Define colors for each category plot
colors = sns.color_palette("hsv", len(categories))

# Plot each category's uphold rate over time, with the overall rate as a reference
for ax, category, color in zip(axs.flat, categories, colors):
    category_data = df_eda_monthly_category[df_eda_monthly_category['category'] == category]
    sns.lineplot(data=category_data, x='date', y='decision_rate', ax=ax, color=color, linewidth=5)
    sns.lineplot(data=df_eda_monthly, x='date', y='decision_rate', ax=ax, color='white', linewidth=5, linestyle='--', label='Overall Uphold Rate')
    ax.set_title(f'Category {category}')
    ax.set_xlabel('Date')
    ax.set_ylabel('Uphold Rate')
    ax.grid(True)  # Add grid to each subplot
    for label in ax.get_xticklabels():
        label.set_rotation(45)
    ax.legend(prop={'size': 20})  # Adjust legend font size for each subplot

# Hide any unused subplots
for i in range(len(categories), len(axs.flat)):
    fig.delaxes(axs.flat[i])

# Set the main title and adjust layout to avoid overlapping
plt.tight_layout(rect=[0, 0, 1, 0.93])
fig.suptitle('Uphold Rate by Product Category Over Time (Monthly)', fontsize=40)

# Specify the save path (user should adjust the path as needed)
save_path = 'path/to/save/uphold_rate_by_product_category_over_time_monthly.png'

# Save the plot as an image file
plt.savefig(save_path)

# Display the plot
plt.show()

## Specific Product Categories

In [ ]:
def analyze_category(df, category):
    """
    Analyzes and visualizes the complaint data for a specific insurance product category.

    This function filters the DataFrame by the specified category, calculates the total frequency 
    of each complaint reason, and generates visualizations for the total frequency of reasons 
    and their distribution over time.

    Parameters:
    df (pd.DataFrame): The input DataFrame containing complaint data.
    category (str): The insurance product category to analyze (e.g., 'c1', 'c2').

    Returns:
    tuple: A tuple containing:
        - df_category (pd.DataFrame): Filtered DataFrame for the specified category.
        - reason_totals (pd.DataFrame): DataFrame with total counts of each complaint reason.
        - df_reason_counts_daily (pd.DataFrame): DataFrame with the daily counts of reasons over time.
    """

    # Ensure the date column is in datetime format
    df['date'] = pd.to_datetime(df['date'])
    
    # Filter the DataFrame by the specified category
    df_category = df[df['category'] == category]
    
    # List of complaint reasons
    reasons = ['reason_1', 'reason_2', 'reason_3', 'reason_4', 'reason_5', 'reason_6', 'reason_7']
    
    # Calculate the total frequency of each reason
    reason_totals = df_category[reasons].sum().reset_index()
    reason_totals.columns = ['reason', 'total_count']
    
    # Adjust font sizes for plots
    plt.rcParams.update({
        'axes.titlesize': 50,  # Size of the plot titles
        'axes.labelsize': 45,  # Size of the axis labels
        'xtick.labelsize': 40,  # Size of the x-axis tick labels
        'ytick.labelsize': 40,  # Size of the y-axis tick labels
        'legend.fontsize': 40,  # Size of the legend font
        'legend.title_fontsize': 40  # Size of the legend title font
    })
    
    # Visualize the total frequency of each reason
    plt.figure(figsize=(25, 20))
    sns.barplot(data=reason_totals, x='reason', y='total_count', linewidth=6)
    plt.title(f'Total Frequency of Each Reason for Category {category}', pad=30)
    plt.xlabel('Reason')
    plt.ylabel('Total Count')
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout(rect=[0, 0, 0.97, 0.97])
    
    # Save the bar plot figure to a file (user should adjust the path)
    save_path = f'path/to/save/total_frequency_of_each_reason_for_category_{category}.png'
    plt.savefig(save_path, bbox_inches='tight')
    plt.show()
    
    # Prepare data for visualizing the frequency of reasons over time
    df_reason_counts = df_category.melt(id_vars=['date'], value_vars=reasons, var_name='reason', value_name='count')
    df_reason_counts['count'] = df_reason_counts['count'].astype(int)
    df_reason_counts_daily = df_reason_counts.groupby([pd.Grouper(key='date', freq='M'), 'reason'], observed=True).sum().reset_index()
    
    # Visualize the frequency of reasons over time
    plt.figure(figsize=(25, 20))
    sns.lineplot(data=df_reason_counts_daily, x='date', y='count', hue='reason', linewidth=6)
    plt.title(f'Frequency of Reasons Over Time for Category {category}', pad=30)
    plt.xlabel('Date')
    plt.ylabel('Count')
    plt.grid(True)
    plt.legend(title='Reason', bbox_to_anchor=(1.05, 1), loc='upper left', prop={'size': 30}, title_fontsize=35)
    plt.xticks(rotation=45)
    plt.tight_layout(rect=[0, 0, 0.97, 0.97])
    
    # Save the line plot figure to a file (user should adjust the path)
    save_path = f'path/to/save/frequency_of_reasons_over_time_for_category_{category}.png'
    plt.savefig(save_path, bbox_inches='tight')
    plt.show()
    
    return df_category, reason_totals, df_reason_counts_daily

### Products already available in SB

In [ ]:
category = 'c3'
df_category, reason_totals, df_reason_counts_daily = analyze_category(df_eda, category)

In [ ]:
category = 'c4'
df_category, reason_totals, df_reason_counts_daily = analyze_category(df_eda, category)

In [ ]:
category = 'c5'
df_category, reason_totals, df_reason_counts_daily = analyze_category(df_eda, category)

In [ ]:
category = 'c6'
df_category, reason_totals, df_reason_counts_daily = analyze_category(df_eda, category)

In [ ]:
category = 'c9'
df_category, reason_totals, df_reason_counts_daily = analyze_category(df_eda, category)

In [ ]:
category = 'c10'
df_category, reason_totals, df_reason_counts_daily = analyze_category(df_eda, category)

In [ ]:
category = 'c11'
df_category, reason_totals, df_reason_counts_daily = analyze_category(df_eda, category)

### Products that SB doesn't have

In [ ]:
category = 'c1'
df_category, reason_totals, df_reason_counts_daily = analyze_category(df_eda, category)

In [ ]:
category = 'c2'
df_category, reason_totals, df_reason_counts_daily = analyze_category(df_eda, category)

In [ ]:
category = 'c7'
df_category, reason_totals, df_reason_counts_daily = analyze_category(df_eda, category)

In [ ]:
category = 'c8'
df_category, reason_totals, df_reason_counts_daily = analyze_category(df_eda, category)